# Exercise 5.3
_______

In [1]:
import numpy as np
import scipy.stats, scipy.special
from matplotlib import pyplot as plt

# plot preset
FONTSIZE = 12
plt.rcParams['font.size'] = FONTSIZE
plt.rcParams['xtick.labelsize'] = FONTSIZE
plt.rcParams['ytick.labelsize'] = FONTSIZE
plt.rcParams['legend.fontsize'] = FONTSIZE
plt.rcParams['axes.titlesize'] = FONTSIZE
plt.rcParams['axes.labelsize'] = FONTSIZE
plt.rcParams['figure.autolayout'] = True
plt.rcParams['figure.dpi'] = 100
plt.rcParams['lines.linewidth'] = 1.5
plt.rcParams['figure.figsize'] = (8, 5)


We draw n=120 elements from an exponential distribution where $\tau$ itself is drawn from a Gamma distribution with $a=4, b=2$ and repeat this $N=10000$ times. In b) we then check for how many of the $N$ samples $\tau$ actually lies within the $90$% confidence intervall (see slide 359).
In order to not have to save all the samples, it makes sense to calculate the necessary information for exercise b) and c) already when we loop over the sample calculation!
Remeber that in the lexture the Gamma distribution was defined as and compare that this is consistent with your implementation:
$$Ga(x;a,b)=\frac{x^{a-1}e^{-x/b}}{b^a\Gamma(a)}$$


In [1]:
# setup 
N = 10000
n = 120
a = 4
b = 2

In [ ]:
# we can sample the N taus already before drawing the N samples and plot them in a histogram
sampled_taus = #TODO 

plt.hist(sampled_taus,bins=100)
plt.xlabel(r'$\tau$')
plt.title(r"Distribution of $\tau s$ from $\Gamma(a=4,b=2)$")
plt.show()

In [ ]:
# now generate the N samples from the exponential distribution from your array of taus and calculate the necessary quantities for b) and c)!

taus_in_ci = 0                      # counter for the times where the true tau lies in the 90% confidence interval
differences = np.zeros(N)   # difference of the sample mean to the true value of tau

for i in range(N):
    sample_exp = #TODO: generate the single sample with tau from your array
    
    mean = np.mean(sample_exp)
    differences[i] = mean - sampled_taus[i]
    
    ciboundaries = #TODO: calculate the boundaries of the 90% CIs
    if #TODO: is the true tau in the CI? If yes, increase the counter.
        taus_in_ci += 1       
        

print("Number of taus inside confidence interval: ",taus_in_ci, "(" , 100 * taus_in_ci / (N), "%)" )

In [3]:
# Use the differences array from above and choose a graphical presentation to show that the values are symmetrically distributed around 0
